# MSc Project : analysis 02 complete data

### Setting up the environment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4
# import pandas as pd
import dask
# from mpl_toolkits.basemap import Basemap
# from datetime import datetime
# from datetime import timedelta
# from scipy import stats
# import scipy.interpolate as interp
%matplotlib inline

In [2]:
from dask.distributed import Client

In [3]:
from functions import ccplot, binning

In [4]:
import seaborn as sns
# Use seaborn style defaults and set default figure size
# plt.style.use('seaborn-pastel')
# sns.set_theme(style="ticks")
plt.style.use(['science', 'notebook'])

In [5]:
# Colormap selection
xr.set_options(cmap_divergent='RdBu', cmap_sequential='YlGnBu')

In [6]:
# client = Client(n_workers=12, threads_per_worker=1, memory_limit='16GB')

### Impoting and preparing data

In [7]:
mfdata_DIR = './data/GPM_lowres_data/*.nc'

gpm = xr.open_mfdataset(mfdata_DIR, chunks=dict(time=8000, lat=40, lon=40), engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# gpm = xr.open_mfdataset(mfdata_DIR, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# gpm = xr.open_dataset('./data/GPM_data/gpm_2000.nc', chunks={"time":2000})

In [8]:
gpm = gpm.unify_chunks()
gpm.chunks

Frozen({'time': (8000, 2272, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1568, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1568, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1568, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1568, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1520, 8000, 8000, 1568, 8000, 688), 'lat': (40, 40, 40, 40, 1), 'lon': (40, 40, 40, 40, 1)})

In [9]:
# precip = regridder(gpm.precipitationCal)

In [10]:
# gpm = gpm.persist()

In [11]:
gpm

,Array,Chunk
Bytes,35.69 GiB,48.83 MiB
Shape,"(369600, 161, 161)","(8000, 40, 40)"
Count,3222 Tasks,1600 Chunks
Type,float32,numpy.ndarray


In [ ]:
precip = gpm.precipCal.resample(time = '1H').mean()

In [15]:
precip = precip.chunk(dict(time=8000, lat=40, lon=40))

In [16]:
precip = precip.unify_chunks()
precip.chunks

((8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  8000,
  800),
 (40, 40, 40, 40, 1),
 (40, 40, 40, 40, 1))

In [17]:
precip

,Array,Chunk
Bytes,17.84 GiB,48.83 MiB
Shape,"(184800, 161, 161)","(8000, 40, 40)"
Count,18483822 Tasks,600 Chunks
Type,float32,numpy.ndarray


In [18]:
# ax = plt.axes(projection=ccrs.platecarree())
# ax.set_extent([59.9, 100.1, 0, 40.1], crs=ccrs.platecarree())
# precip.isel(time=0).plot.contourf(ax = ax, extend='neither', levels=15)
# ax.coastlines()
# gridliner = ax.gridlines(crs=ccrs.platecarree(), draw_labels=true, linewidth=2, color='gray', alpha=0.1, linestyle='--')
# gridliner.top_labels = false
# gridliner.right_labels = false
# gridliner.ylines = false  # you need false
# gridliner.xlines = false  # you need false

In [19]:
mfdata_DIR2 = './data/era_data/*.nc'

era = xr.open_mfdataset(mfdata_DIR2, chunks=dict(time=5000, lat=40, lon=40), engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [20]:
era = era.unify_chunks()
era.chunks

Frozen({'time': (5000, 3784, 5000, 3760, 5000, 3760, 5000, 3760, 5000, 3784, 5000, 3760, 5000, 3760, 5000, 3760, 5000, 3784, 5000, 3760, 5000, 3760, 5000, 3760, 5000, 3784, 5000, 3760, 5000, 3760, 5000, 3760, 5000, 3784, 5000, 3760, 5000, 3760, 5000, 3760, 5000, 3784, 5000, 1775), 'latitude': (161,), 'longitude': (161,), 'expver': (2,)})

In [21]:
era = era.sel(expver=1, drop=True)

In [22]:
era = era.transpose('time', 'latitude', 'longitude')

In [23]:
era = era.rename_dims({'longitude':'lon', 'latitude':'lat'})

In [24]:
era = era.rename({'longitude':'lon', 'latitude':'lat'})

In [25]:
era = era.sel(time = slice("2000-06-01 00:00:00","2021-06-30 23:00:00"))

In [26]:
era

<xarray.Dataset>
Dimensions:  (lon: 161, lat: 161, time: 184800)
Coordinates:
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float32 40.0 39.75 39.5 39.25 39.0 ... 1.0 0.75 0.5 0.25 0.0
  * time     (time) datetime64[ns] 2000-06-01 ... 2021-06-30T23:00:00
Data variables:
    u10      (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    v10      (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    d2m      (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    t2m      (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    mx2t     (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    mn2t     (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
    sst      (time, lat, lon) float32 dask.array<chunksize=(1352, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-10-02 12:58:09 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [27]:
t2m = era.t2m
d2m = era.d2m

In [28]:
t2m = t2m.chunk(dict(time=8000, lat=40, lon=40))
d2m = d2m.chunk(dict(time=8000, lat=40, lon=40))
t2m

,Array,Chunk
Bytes,17.84 GiB,48.83 MiB
Shape,"(184800, 161, 161)","(8000, 40, 40)"
Count,2533 Tasks,600 Chunks
Type,float32,numpy.ndarray


In [29]:
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_extent([59.9, 100.1, 0, 40.1], crs=ccrs.PlateCarree())
# t2m.isel(time=0).plot.contourf(ax = ax, cmap='turbo', extend='neither', levels=15)
# ax.coastlines()
# gridliner = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.1, linestyle='--')
# gridliner.top_labels = False
# gridliner.right_labels = False
# gridliner.ylines = False  # you need False
# gridliner.xlines = False  # you need False

In [30]:
# ds_comb = xr.merge([precip, t2m, d2m])

In [31]:
# del precip
# del t2m
# del d2m
# del gpm
# del era

In [32]:
# ds_comb

## Binning

In [35]:
# # Plotting the region
# # fig = plt.figure(figsize=(6,6))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_extent([59.9, 100.1, -0.1, 40.1], crs=ccrs.PlateCarree())
# # marking wcgw
# ax.plot([72, 72], [8, 18.75], 'k', ls = '--', lw = 0.9)
# ax.plot([77.75, 77.75], [8, 18.75], 'k', ls = '--', lw = 0.9)
# ax.plot([72, 77.75], [8, 8], 'k', ls = '--', lw=0.9)
# ax.plot([72, 77.75], [18.75, 18.75], 'k', ls = '--', lw = 0.9)
# # marking sip region
# ax.plot([77.5, 77.5], [8, 18.75], 'k', ls = '--', lw = 0.9)
# ax.plot([84, 84], [8, 18.75], 'k', ls = '--', lw = 0.9)
# ax.plot([77.75, 84], [8, 8], 'k', ls = '--', lw=0.9)
# ax.plot([77.75, 84], [18.75, 18.75], 'k', ls = '--', lw = 0.9)
# # marking cir region
# ax.plot([76, 76], [19, 26], 'k', ls = '--', lw = 0.9)
# ax.plot([86, 86], [19, 26], 'k', ls = '--', lw = 0.9)
# ax.plot([76, 86], [19, 19], 'k', ls = '--', lw=0.9)
# ax.plot([76, 86], [26, 26], 'k', ls = '--', lw = 0.9)
# precip.sel(time="2019-07-20 09:00:00").plot.contourf(ax = ax, extend='neither', levels=15)
# # ax.coastlines(alpha=0.7)
# gridliner = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.05, linestyle='--')
# # ax.add_feature(cfeature.BORDERS, alpha=0.7)
# ax.add_feature(cfeature.COASTLINE, alpha=1.0)
# gridliner.top_labels = False
# gridliner.right_labels = False
# gridliner.ylines = False  # you need False
# gridliner.xlines = False  # you need False
# # ax.tick_params(axis='both', labelsize=12)
# plt.savefig('./images/region.pdf', dpi=300)

In [36]:
ds_comb = xr.merge([precip.sel(time=slice("2000-06-01 00:00:00","2004-12-31 23:00:00")), t2m.sel(time=slice("2000-06-01 00:00:00","2004-12-31 23:00:00")), d2m.sel(time=slice("2000-06-01 00:00:00","2004-12-31 23:00:00"))])

In [37]:
ds_comb

<xarray.Dataset>
Dimensions:    (lon: 161, lat: 161, time: 40200)
Coordinates:
  * lon        (lon) float64 60.0 60.25 60.5 60.75 ... 99.25 99.5 99.75 100.0
  * lat        (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.25 39.5 39.75 40.0
  * time       (time) datetime64[ns] 2000-06-01 ... 2004-12-31T23:00:00
Data variables:
    precipCal  (time, lat, lon) float32 dask.array<chunksize=(8000, 40, 40), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(8000, 40, 40), meta=np.ndarray>
    d2m        (time, lat, lon) float32 dask.array<chunksize=(8000, 40, 40), meta=np.ndarray>

In [38]:
mon = [6,7,8,9]
premon = [3,4,5]
postmon = [10,11,12]
wint = [1,2]

In [39]:
# for monsoon season only first

In [40]:
ds_comb_mon = xr.concat([list(ds_comb.groupby('time.month'))[i-1][1] for i in mon], dim='time')

In [41]:
ds_comb_mon

,Array,Chunk
Bytes,1.41 GiB,4.54 MiB
Shape,"(14640, 161, 161)","(744, 40, 40)"
Count,18484972 Tasks,500 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,1.41 GiB,4.54 MiB
Shape,"(14640, 161, 161)","(744, 40, 40)"
Count,3683 Tasks,500 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


### analysing

In [42]:
precip = ds_comb_mon.precipCal
t2m = ds_comb_mon.t2m
d2m = ds_comb_mon.d2m

In [43]:
precip_daily_rate = precip[::-1].rolling(time=24).sum()[::-1]

In [44]:
# Set values less than 1 mm/hr to nan

precip = xr.where(precip_daily_rate > 1, precip, np.nan).chunk(dict(time=-1, lat=40, lon=40))
# precip = xr.where(precip > 0.1, precip, np.nan).chunk(dict(time=-1, lat=40, lon=40))
precip = precip.where(precip > 0).chunk(dict(time=-1, lat=40, lon=40))
t2m = t2m.where(precip != np.nan).chunk(dict(time=-1, lat=40, lon=40))
d2m = d2m.where(precip != np.nan).chunk(dict(time=-1, lat=40, lon=40))

In [45]:
ind_precip = precip.chunk(dict(time=-1, lat=40, lon=40))
ind_t2m = t2m.chunk(dict(time=-1, lat=40, lon=40))
ind_d2m = d2m.chunk(dict(time=-1, lat=40, lon=40))

In [3]:
ccscale95_t2m_slope, ccscale95_t2m_p, ccscale95_d2m_slope, ccscale95_d2m_p  = binning.get_binned_3d(ind_precip, ind_t2m, ind_d2m, percentile_val=0.95, bin_nr=20)

NameError: name 'binning' is not defined

In [ ]:
# ccscale50_t2m_slope, ccscale50_t2m_p, ccscale50_d2m_slope, ccscale50_d2m_p  = binning.get_binned_3d(ind_precip, ind_t2m, ind_d2m, percentile_val=0.50, bin_nr=20)

## Plotting

In [ ]:
# # Plotting function -- Fpr India
# def ccplot_3d(slope_da, r_da, extent_list, title, threshold_sig=0.95, marker_size=2, **kwargs):
#     """
#     Function to plot the output of binning 3d function
#     -----
#     inputs are -
#     slope_da : datarray containing slope
#     r_da : datarray containing R^2 values for goodness of fit
#     title : The title of the output plot
#     extent_list : [59.9, 100.1, -0.1, 40.1]
#     """
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.set_extent(extent_list, crs=ccrs.PlateCarree())
#     (100*(np.exp(slope_da) - 1)).plot.contourf(ax = ax, cbar_kwargs={"label":"C-C scale"}, **kwargs)
# #     (100*(np.exp(slope_da) - 1)).plot.pcolormesh(ax = ax, cbar_kwargs={"label":"C-C scale"}, **kwargs)
#     x, y = np.meshgrid(slope_da.coords['lon'], slope_da.coords['lat'])
#     plt.scatter(x[(np.abs(r_da.to_numpy()) > threshold_sig)],y[(np.abs(r_da.to_numpy()) > threshold_sig)], marker='o', color = 'k', s=marker_size)
#     gridliner = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.05, linestyle='--')
#     # ax.coastlines(alpha=0.7)
#     # ax.add_feature(cfeature.BORDERS, alpha=0.7)
#     ax.add_feature(cfeature.COASTLINE, alpha=1.0)
#     gridliner.top_labels = False
#     gridliner.right_labels = False
#     gridliner.ylines = False  # you need False
#     gridliner.xlines = False  # you need False
#     ax.set_title(title)
#     # ax.set_xlabel('Latitude')
#     # ax.set_ylabel('Longitude')
#     # ax.tick_params(axis='both', labelsize=12)
# # ax.tick_params(axis='both', labelsize=12)

In [1]:
# Extent_lists
ind = [59.9, 100.1, -0.1, 40.1]
sip = [77.65, 84.1, 7.9, 18.85]
wcwg = [71.9, 77.15, 7.9, 18.85]
cir = [75.9, 86.1, 18.9, 26.1]

In [2]:
# India T2m 99th
ccplot_3d(ccscale95_t2m_slope, ccscale95_t2m_p, ind, 'T2m 95th p.', threshold_sig=0.8, marker_size=.02, levels=20, extend='both', vmax=20, center=0.0)
plt.savefig('./images/india_t2m95.pdf', dpi=300)

NameError: name 'ccplot_3d' is not defined

In [ ]:
# India D2m 99th
ccplot_3d(ccscale95_d2m_slope, ccscale95_d2m_p, ind, 'D2m 95th p.', threshold_sig=0.8, marker_size=.02, levels=20, extend='both', vmax=50, center=0.0)
plt.savefig('./images/india_d2m95.pdf', dpi=300)

## SIP region

In [ ]:
ccplot_3d(ccscale95_t2m_slope.sel(lat =slice(8,19), lon = slice(77.75,84)), ccscale95_t2m_p.sel(lat =slice(8,19), lon = slice(77.75,84)), sip, 'SIP T2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=80, center=0.0)
plt.savefig('./images/sip_t2m95.pdf', dpi=300)

In [ ]:
ccplot_3d(ccscale95_d2m_slope.sel(lat =slice(8,19), lon = slice(77.75,84)), ccscale95_d2m_p.sel(lat =slice(8,19), lon = slice(77.75,84)), sip, 'SIP D2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=80, center=0.0)
plt.savefig('./images/sip_d2m95.pdf', dpi=300)

## WCWG region

In [ ]:
ccplot_3d(ccscale95_t2m_slope.sel(lat =slice(8,18.75), lon = slice(72,77.5)), ccscale95_t2m_p.sel(lat =slice(8,18.75), lon = slice(72,77.5)), wcwg, 'WCWG T2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=50, center=0.0)
plt.savefig('./images/wcwg_t2m95.pdf', dpi=300)

In [ ]:
ccplot_3d(ccscale95_d2m_slope.sel(lat =slice(8,18.75), lon = slice(72,77.5)), ccscale95_d2m_p.sel(lat =slice(8,18.75), lon = slice(72,77.5)), wcwg, 'WCWG D2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=90, center=0.0)
plt.savefig('./images/wcwg_d2m95.pdf', dpi=300)

## CIR Region

In [ ]:
ccplot_3d(ccscale95_t2m_slope.sel(lat =slice(19,26), lon = slice(76,86)), ccscale95_t2m_p.sel(lat =slice(19,26), lon = slice(76,86)), cir, 'CIR T2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=20, center=0.0)
plt.savefig('./images/cir_t2m95.pdf', dpi=300)

In [ ]:
ccplot_3d(ccscale95_d2m_slope.sel(lat =slice(19,26), lon = slice(76,86)), ccscale95_d2m_p.sel(lat =slice(19,26), lon = slice(76,86)), cir, 'CIR D2m 95th p.', threshold_sig=0.8, marker_size=2, levels=20, extend='both', vmax=40, center=0.0)
plt.savefig('./images/cir_d2m95.pdf', dpi=300)